In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [116]:
btc_data = pd.read_csv('btc_1h_data_with_ma.csv', index_col=False)

In [97]:
ma_candidates = [10, 20, 30, 50, 60, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 200]

In [98]:
# Function to backtest strategy and calculate cumulative profit
def backtest_and_calculate_profit(df, ma):
    df['Position'] = np.where(df['close'] > df[f'MA_{ma}'], 1, 0)  # 1 for buy, 0 for sell
    df['Signal'] = df['Position'].diff()  # Buy signal when diff > 0, sell signal when diff < 0

    # Initial values
    initial_cash = 10000  # Starting with $10,000
    btc_holding = 0
    cash = initial_cash
    purchase_price = 0
    stop_loss = 0 # No stop loss    
    
    # stop_loss = 0.995 # 0.5% loss
    stop_loss = 0.99 # 1% loss
    # stop_loss = 0.98 # 2% loss
    # stop_loss = 0.97 # 3% loss

    for i in range(0, len(df)):
        price = df['close'].iloc[i]
        # Stop loss check: Sell if the price drops more than 1% below the purchase price
        if btc_holding > 0 and price < purchase_price * stop_loss:
            cash = btc_holding * price  # Sell all BTC and hold cash
            btc_holding = 0  # No BTC left after selling
            current_time = df['timestamp'].iloc[i]
            print(f"{current_time}: Stop loss triggered! Sold BTC at {price:.2f}, now holding {cash:.2f} USDT")

        else:
            # Buy signal
            if df['Signal'].iloc[i] == 1:  # Crosses above MA
                if cash > 0:
                    btc_holding = cash / price  # Buy BTC with all available cash
                    cash = 0  # No cash left after buying
                    purchase_price = price
                    current_time = df['timestamp'].iloc[i]
                    print(f"{current_time}: Bought BTC at {price:.2f}, holding {btc_holding:.4f} BTC")


            # Sell signal
            elif df['Signal'].iloc[i] == -1:  # Crosses below MA
                if btc_holding > 0:
                    cash = btc_holding * price  # Sell all BTC and hold cash
                    btc_holding = 0  # No BTC left after selling
                    current_time = df['timestamp'].iloc[i]
                    print(f"{current_time}: Sold BTC at {price:.2f}, now holding {cash:.2f} USDT\n")
        
        # Calculate current portfolio value (cash + value of BTC)
        portfolio_value = cash + (btc_holding * price)
        # Update the portfolio value for each iteration
        df.loc[i, 'portfolio_value'] = portfolio_value

    # Drop 'Signal' and 'Position' columns
    df = df.drop(columns=['Signal', 'Position'])

    # df['portfolio_value'] = portfolio_value
    print(f"Final value for MA {ma}: {portfolio_value:,.2f} USDT")
    return df

In [117]:
# Filter btc_data for the year 2018 and 2022
btc_data = btc_data[pd.to_datetime(btc_data['timestamp']).dt.year.isin([2018, 2022])]
btc_data = btc_data.drop(btc_data[~pd.to_datetime(btc_data['timestamp']).dt.year.isin([2018, 2022])].index)
btc_data = btc_data.reset_index(drop=True)

In [ ]:
btc_data.tail()

In [ ]:
profit_df = backtest_and_calculate_profit(btc_data, 100)
# print(profit_df['Cumulative Profit'])

## Test all ma candidates

In [119]:
portfolio_value_by_ma = {}

In [ ]:
for ma in ma_candidates:
    profit = backtest_and_calculate_profit(btc_data, ma)
    portfolio_value_by_ma[ma] = profit['portfolio_value'].iloc[-1]

In [ ]:
for ma, value in portfolio_value_by_ma.items():
    print(f"Final value for MA {ma}: {value:,.2f} USDT") 

In [ ]:
# Find the MA with the highest portfolio value
best_ma = max(portfolio_value_by_ma, key=portfolio_value_by_ma.get)
best_value = portfolio_value_by_ma[best_ma]

print(f"The best performing MA is {best_ma} with a final portfolio value of {best_value:,.2f} USDT")


In [ ]:
# Sort the dictionary by MA values
sorted_portfolio_value = dict(sorted(portfolio_value_by_ma.items()))

# Extract MA values and corresponding portfolio values
ma_values = list(sorted_portfolio_value.keys())
portfolio_values = list(sorted_portfolio_value.values())

# Create the plot
plt.figure(figsize=(12, 6))
plt.plot(ma_values, portfolio_values, marker='o')

# Customize the plot
plt.title('Portfolio Value by Moving Average Period', fontsize=16)
plt.xlabel('MA', fontsize=12)
plt.ylabel('Final Portfolio Value (USDT)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)

# Add value labels on top of each point
for i, value in enumerate(portfolio_values):
    plt.text(ma_values[i], value, f'{value:,.0f}', ha='center', va='bottom')

# Add x-axis labels for each MA value
plt.xticks(ma_values, [f'{ma}' for ma in ma_values], rotation=0, ha='right')

# Adjust the bottom margin to accommodate the labels
plt.subplots_adjust(bottom=0.2)

# Add a horizontal line at 145,580.15
plt.axhline(y=145580.15, color='red', linestyle='--', label='Hold BTC Value')
plt.legend()

# Adjust y-axis to start from 0
plt.ylim(bottom=0)

# Show the plot
plt.tight_layout()
plt.show()


## Just holding BTC


In [57]:
# Function to calculate final value of holding BTC
def calculate_holding_value(df):
    initial_cash = 10000  # Start with $10,000 in cash

    # Buy BTC at the first available price
    initial_price = df['close'].iloc[0]
    btc_holding = initial_cash / initial_price  # Buy BTC with all available cash

    # Sell BTC at the last available price
    final_price = df['close'].iloc[-1]
    final_value = btc_holding * final_price  # Calculate the value of the BTC holding

    print(f"Initial BTC price: {initial_price:.2f} USDT")
    print(f"Final BTC price: {final_price:.2f} USDT")
    print(f"Final value of holding BTC: {final_value:,.2f} USDT")
    
    return final_value

In [ ]:

# Example usage
# Assuming 'df' is a DataFrame with your historical price data (e.g., 1-hour BTC/USDT prices)
final_value = calculate_holding_value(btc_data)

## Portfolio cumulative profit

In [ ]:
# Create a new figure with a specific size
plt.figure(figsize=(12, 6))

# Import matplotlib.dates for date handling
import matplotlib.dates as mdates

# Plot the portfolio value over time
plt.plot(btc_data['timestamp'], btc_data['portfolio_value'], label='Portfolio Value')

# Set x-axis ticks to show only at one month intervals
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # Set tick every 1 month
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Format the date

# Hide unnecessary minor ticks
plt.gca().xaxis.set_minor_locator(mdates.MonthLocator())
plt.gca().xaxis.set_tick_params(which='minor', bottom=False)

# Remove unnecessary rotation for x-axis labels
plt.gca().xaxis.set_tick_params(rotation=45)  # Rotate only the major ticks for readability

# Set labels and title
plt.xlabel('Date')
plt.ylabel('Portfolio Value (USDT)')
plt.title('Portfolio Value Over Time')

# Add legend
plt.legend()

# Ensure y-axis starts from 0
plt.ylim(bottom=0)

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()